In [ ]:
!pip install geopy

In [ ]:
import streamlit as st
import pandas as pd
from geopy.geocoders import Nominatim

# Initialize the geocoder
geolocator = Nominatim(user_agent="tipscout")

# Function to get coordinates
def get_coordinates(address):
    location = geolocator.geocode(address)
    return location.latitude, location.longitude

# Example address
address = "Nørrebrogade 59, 2200 Copenhagen N"

# Get the coordinates
latitude, longitude = get_coordinates(address)

# Create DataFrame with the coordinates
data = {'latitude': [latitude], 'longitude': [longitude]}
df = pd.DataFrame(data)

df
# Show map
# st.map(df)


In [ ]:
from airflow.dags.include.scraper.scraper import TipsterScraper
import pandas as pd

def scrape_data() -> pd.DataFrame:
    scraper = TipsterScraper(main_url="https://www.tipster.io/team")
    return scraper.scrape()

deals_df = scrape_data()

deals_df

In [ ]:
deals_df

In [ ]:
import requests

response = requests.get("https://www.tipster.io/")
print(response.status_code)  # 200 indicates success
print(response.text)  # Check if HTML content is properly returned

In [ ]:
# find all the links on "https://www.tipster.io/
from bs4 import BeautifulSoup

# Parse the HTML content
soup = BeautifulSoup(response.text, "html.parser")

# Find all the links on the page
links = soup.find_all("a")

# Print the links
for link in links:
    print(link.get("href"))

In [ ]:
import pytz
from datetime import datetime
from timezonefinder import TimezoneFinder

# Sample inputs
iso_country_code = 'DK'  # Example ISO country code
utc_timestamp = '2024-10-31T14:30:00Z'  # Example UTC timestamp

# Convert the UTC timestamp to a datetime object
utc_time = datetime.strptime(utc_timestamp, '%Y-%m-%dT%H:%M:%SZ')
utc_time = utc_time.replace(tzinfo=pytz.UTC)

# Initialize TimezoneFinder and get the country's time zone (you may need latitude/longitude)
tz_finder = TimezoneFinder()
# Sample: For a more accurate result, use latitude and longitude of the transaction location
# timezone_str = tz_finder.timezone_at(lat=latitude, lng=longitude)

# For simplicity, let's assume you know the country's time zone
timezone_str = 'CET'  # Example: Eastern Time Zone

# Convert the UTC time to local time
local_tz = pytz.timezone(timezone_str)
local_time = utc_time.astimezone(local_tz)

print(f"Local time of the transaction: {local_time}")


In [ ]:
import re

def clean_location_data(raw_html: str) -> str:
    """
    Cleans location data from HTML content using regex.

    :param raw_html: Raw HTML string containing location information
    :return: Cleaned string with addresses comma-separated and multiple locations newline-separated
    """
    # Step 1: Replace <br><br> with a newline, indicating a new location
    cleaned_data = re.sub(r'<br>\s*<br>', '\n', raw_html)

    # Step 2: Replace remaining <br> with a comma, indicating part of the same address
    cleaned_data = re.sub(r'<br>', ', ', cleaned_data)

    # Step 3: Remove any extra spaces before/after commas and newlines
    cleaned_data = re.sub(r'\s*,\s*', ', ', cleaned_data)
    cleaned_data = cleaned_data.strip()

    # Step 4: Add quotes around each address by splitting on newlines
    addresses = [f'"{address.strip()}"' for address in cleaned_data.split('\n')]

    # Step 5: Join the addresses back together as a comma-separated string
    return ', '.join(addresses)

# Example usage
raw_html = """
<p>Dybbølsgade 55<br> 1721 Copenhagen V<br><br>Rosenvængets Allé 3<br> 2100 Copenhagen Ø<br><br>ILLUM UNDERGROUND<br> Østergade 52<br> 1001 Copenhagen K</p>
"""
print(clean_location_data(raw_html))

In [ ]:
from airflow.dags.include.scraper.scraper.TipsterScraper import _get_location_info

In [ ]:
response = requests.get(self.main_url)
page_soup = BeautifulSoup(response.text, "html.parser")

def get_location_info(page_soup) -> str:
    """Extract the location information of the deal as a string.

    :param page_soup: BeautifulSoup object of the deal's page
    :return: Comma-separated, quote-encased string of complete addresses
    """
    # Find the accordion item for "WHERE"
    where_section = page_soup.find("span", text="+ WHERE")
    print(where_section)
    cleaned_data = re.sub(r'<br>\s*<br>', '\n', where_section)

    # Step 2: Replace remaining <br> with a comma, indicating part of the same address
    cleaned_data = re.sub(r'<br>', ', ', cleaned_data)

    # Step 3: Remove any extra spaces before/after commas and newlines
    cleaned_data = re.sub(r'\s*,\s*', ', ', cleaned_data)
    cleaned_data = cleaned_data.strip()
    print(cleaned_data)

    # Create a comma-separated, quote-encased string of complete addresses
    return ", ".join([f'"{addr}"' for addr in cleaned_data])

In [ ]:
import re
from bs4 import BeautifulSoup
import requests

def get_location_info(page_soup) -> str:
    """Extract the location information of the deal as a string.

    :param page_soup: BeautifulSoup object of the deal's page
    :return: Comma-separated, quote-encased string of locations
    """
    # Find the accordion item for "WHERE"
    where_section = page_soup.find("span", text="+ WHERE")
    locations = []
    if where_section:
        where_content = where_section.find_next("div", class_="accordion__content")
        if where_content and where_content.p:
            # Clean the HTML to extract plain text
            clean_text = BeautifulSoup(where_content.p.decode_contents(), "html.parser").get_text()
            # Split the locations by double line breaks and strip whitespace
            raw_locations = clean_text.split('\n\n')
            for loc in raw_locations:
                # Split each address by a single line break and strip whitespace
                addresses = [address.strip() for address in loc.split('\n') if address.strip()]
                # Join the address parts with commas
                full_address = ', '.join(addresses)
                if full_address:
                    locations.append(f'"{full_address}"')

    # Create a comma-separated string of locations
    return ', '.join(locations)

# get the html from a specific web page and save to a string


sample_html = requests.get("https://www.tipster.io/team/33945.html")
# print(sample_html)
page_soup = BeautifulSoup(sample_html.text, "html.parser")
# print(page_soup)

result = get_location_info(page_soup)
print(result)

In [ ]:
import re
from bs4 import BeautifulSoup
import requests

def get_location_info(page_soup) -> str:
    """Extract the location information of the deal as a string.

    :param page_soup: BeautifulSoup object of the deal's page
    :return: Comma-separated, quote-encased string of locations
    """
    # Find the accordion item for "WHERE"
    where_section = page_soup.find("span", text="+ WHERE")
    locations = []
    if where_section:
        where_content = where_section.find_next("div", class_="accordion__content")
        if where_content and where_content.p:
            # Get the raw HTML text
            raw_html = where_content.p.decode_contents()
            # Use regex to clean up the HTML
            clean_text = re.sub(r'<br/> ', ', ', raw_html)  # Replace single <br/> with a comma
            clean_text = re.sub(r'<br/><br/>', "', '", clean_text)  # Replace double <br/> with ', '
            clean_text = clean_text.replace('<br>', '')  # Remove any remaining <br>
            # Remove HTML tags and split by comma
            raw_locations = clean_text.split("', '")
            # Strip whitespace and format each address
            locations = [f"'{loc.strip()}'" for loc in raw_locations if loc.strip()]

    # Create a comma-separated string of locations
    return ', '.join(locations)

# Test with sample HTML
sample_html = requests.get("https://www.tipster.io/team/33973.html")
page_soup = BeautifulSoup(sample_html.text, "html.parser")

result = get_location_info(page_soup)
print(result)


In [16]:
# from datetime import datetime
import pandas as pd
from airflow.dags.include.scraper.scraper import TipsterScraper
from airflow.dags.include.database.database import DuckDBHandler


def scrape_data() -> pd.DataFrame:
    scraper = TipsterScraper(main_url="https://www.tipster.io/team")
    return scraper.scrape()

# Database Insertion Task
def store_data(deals_df: pd.DataFrame):
    db_handler = DuckDBHandler("airflow/data/tipsterdeals.duckdb")
    if not db_handler.table_exists("tipsterdeals"):
        db_handler.create_table()
    db_handler.insert_data(deals_df)
    db_handler.close()

# Workflow: scrape -> store
deals_df = scrape_data()
store_data(deals_df)
deals_df

/Users/kilian.tscherny/Documents/Development/tipscout/airflow/dags/include/scraper/scraper.py:123 DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.

/Users/kilian.tscherny/Documents/Development/tipscout/airflow/dags/include/scraper/scraper.py:153 DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.

,surrogate_key,deal_id,full_url,status,sold,remaining,deal_description,date_added,location,hours,merchant_name,old_price,old_currency,new_price,new_currency,inserted_at
0,832c480015e3a6ea4026130cd67ecabc,33973,https://www.tipster.io/team/33973.html,SOLD OUT,2629,0,Pick whatever you like at LÚA – Discover hand-...,2024-09-29,"'Dronningens Tværgade 12, 1302 København K'",Wednesday - Saturday 11:00-21:00\r\n\tSunday 1...,LÚA,200,DKK,100,DKK,2024-10-09 14:49:26
1,c40230e84c637a01d10052c5a5bce71c,33945,https://www.tipster.io/team/33945.html,SOLD OUT,5701,0,Pick any burger combo + lemonade at Eden Jaxx ...,2024-09-25,"'Dybbølsgade 55, 1721 Copenhagen V', 'Rosenvæn...",Vesterbro\r\n\tMonday - Sunday 12:00-20:30\n\r...,Eden Jaxx,218,DKK,109,DKK,2024-10-09 14:49:26
2,b4b896bae30d0af6c24064356186693b,33925,https://www.tipster.io/team/33925.html,SOLD OUT,2002,0,Pick whatever you like at It’s Caramel by Kara...,2024-09-23,"'Jægersborggade 36, 2200 Copenhagen N'",Monday 11:00-17:30\r\n\tTuesday - Friday 10:00...,It’s Caramel,200,DKK,100,DKK,2024-10-09 14:49:26
3,a0e7128e7e8ad541ea6ca36a5410a2bb,33875,https://www.tipster.io/team/33875.html,SOLD OUT,12700,0,Fish and chips + any drink at Hooked – With lo...,2024-09-18,"'Nørrebrogade 59, 2200 Copenhagen N', 'Halmtor...",Monday - Sunday 12:00-22:00\n\r\n\t(the kitche...,Hooked,220,DKK,110,DKK,2024-10-09 14:49:26
4,3682c763600ed35a74cb1ecc5ccfd522,33854,https://www.tipster.io/team/33854.html,SOLD OUT,1500,0,Smørrebrød tasting menu at Lola – A Michelin-r...,2024-09-25,"'Christianshavns Voldgade 50, 1424 Copenhagen K'",Lunch\r\n\tMonday - Saturday 11:30-15:00\n\r\n...,Lola,310,DKK,185,DKK,2024-10-09 14:49:26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0d7c875f4ac2e10e7f2d73bfc3a3d14d,31318,https://www.tipster.io/team/31318.html,SOLD OUT,3544,0,Pick any gyros combo at Strangas in city cente...,2023-11-30,"'Gothersgade 27, 1123 Copenhagen K'",Monday - Sunday 11:00-21:00,Strangas,135,DKK,67,DKK,2024-10-09 14:49:45
96,03d321b8448559a641e083b2bbe20072,31274,https://www.tipster.io/team/31274.html,SOLD OUT,1250,0,Pick any draft beer + pizza slice at ÅBEN in C...,2023-11-28,"'Copenhagen Airport, Terminal 2 (outside airpo...",Monday - Wednesday 10:00-20:00\r\n\tThursday 1...,ÅBEN CPH Airport,170,DKK,85,DKK,2024-10-09 14:49:45
97,cf1147e962848fdfaaa2684bbb92430e,31245,https://www.tipster.io/team/31245.html,SOLD OUT,2018,0,Pick whatever you like at LÚA – Nourishing noo...,2023-11-26,"'Dronningens Tværgade 12, 1302 København K'",Wednesday - Saturday 11:00-21:00\r\n\tSunday 1...,LÚA,200,DKK,100,DKK,2024-10-09 14:49:45
98,4b6310bba1cb020a2b3af78d925149b3,31225,https://www.tipster.io/team/31225.html,SOLD OUT,2258,0,Pick whatever you like at Leafs – Nørrebro fav...,2023-11-22,"'Hørsholmsgade 32, 2200 København N'",Wednesday - Thursday 16:00-24:00\r\n\tFriday -...,Leafs,300,DKK,150,DKK,2024-10-09 14:49:45


In [8]:
import re
import logging
url = "https://www.tipster.io/team/33945.html"
logging.debug(f"Extracting deal ID from URL: {url}")
match = re.search(r"/team/(\d+)\.html$", url)
if match:
    deal_id = match.group(1)
    logging.debug(f"Extracted deal ID: {deal_id}")
    return deal_id
logging.debug("No deal ID found in URL")
return None

SyntaxError: 'return' outside function (363180970.py, line 9)

In [11]:
deals_df

,surrogate_key,deal_id,full_url,status,sold,remaining,deal_description,date_added,location,hours,merchant_name,old_price,old_currency,new_price,new_currency,inserted_at
0,228338ed06c89ed470c78e252b412bff,33973,https://www.tipster.io/team/33973.html,SOLD OUT,2629,0,Pick whatever you like at LÚA – Discover hand-...,2024-09-29,"'Dronningens Tværgade 12, 1302 København K'",Wednesday - Saturday 11:00-21:00\r\n\tSunday 1...,LÚA,200,DKK,100,DKK,2024-10-09 14:09:34
1,6104cd80aaf6ab8f709761aa52d73f5f,33945,https://www.tipster.io/team/33945.html,SOLD OUT,5701,0,Pick any burger combo + lemonade at Eden Jaxx ...,2024-09-25,"'Dybbølsgade 55, 1721 Copenhagen V', 'Rosenvæn...",Vesterbro\r\n\tMonday - Sunday 12:00-20:30\n\r...,Eden Jaxx,218,DKK,109,DKK,2024-10-09 14:09:35
2,5d68ee7cdd93d57aa2d8ea770b7e8bba,33925,https://www.tipster.io/team/33925.html,SOLD OUT,2002,0,Pick whatever you like at It’s Caramel by Kara...,2024-09-23,"'Jægersborggade 36, 2200 Copenhagen N'",Monday 11:00-17:30\r\n\tTuesday - Friday 10:00...,It’s Caramel,200,DKK,100,DKK,2024-10-09 14:09:35
3,39619dcdb3c75b010c036d85a5aeb2b1,33875,https://www.tipster.io/team/33875.html,SOLD OUT,12700,0,Fish and chips + any drink at Hooked – With lo...,2024-09-18,"'Nørrebrogade 59, 2200 Copenhagen N', 'Halmtor...",Monday - Sunday 12:00-22:00\n\r\n\t(the kitche...,Hooked,220,DKK,110,DKK,2024-10-09 14:09:35
4,1d29ae3072525697b57f7f6ee37ccac8,33854,https://www.tipster.io/team/33854.html,SOLD OUT,1500,0,Smørrebrød tasting menu at Lola – A Michelin-r...,2024-09-25,"'Christianshavns Voldgade 50, 1424 Copenhagen K'",Lunch\r\n\tMonday - Saturday 11:30-15:00\n\r\n...,Lola,310,DKK,185,DKK,2024-10-09 14:09:35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,7d728ad795428ea7816c9f9c0445aacc,31318,https://www.tipster.io/team/31318.html,SOLD OUT,3544,0,Pick any gyros combo at Strangas in city cente...,2023-11-30,"'Gothersgade 27, 1123 Copenhagen K'",Monday - Sunday 11:00-21:00,Strangas,135,DKK,67,DKK,2024-10-09 14:09:55
96,e98f5193da427245887961981da5f904,31274,https://www.tipster.io/team/31274.html,SOLD OUT,1250,0,Pick any draft beer + pizza slice at ÅBEN in C...,2023-11-28,"'Copenhagen Airport, Terminal 2 (outside airpo...",Monday - Wednesday 10:00-20:00\r\n\tThursday 1...,ÅBEN CPH Airport,170,DKK,85,DKK,2024-10-09 14:09:55
97,c7489d800a1cceb039d57f9d1f05ea59,31245,https://www.tipster.io/team/31245.html,SOLD OUT,2018,0,Pick whatever you like at LÚA – Nourishing noo...,2023-11-26,"'Dronningens Tværgade 12, 1302 København K'",Wednesday - Saturday 11:00-21:00\r\n\tSunday 1...,LÚA,200,DKK,100,DKK,2024-10-09 14:09:55
98,b607a1cef38217572bd382b9c0af12a2,31225,https://www.tipster.io/team/31225.html,SOLD OUT,2258,0,Pick whatever you like at Leafs – Nørrebro fav...,2023-11-22,"'Hørsholmsgade 32, 2200 København N'",Wednesday - Thursday 16:00-24:00\r\n\tFriday -...,Leafs,300,DKK,150,DKK,2024-10-09 14:09:56


In [20]:
import duckdb

def check_table_name(db_path):
    conn = duckdb.connect(db_path)
    try:
        tables = conn.execute("SHOW TABLES").fetchall()
        return tables
    finally:
        conn.close()

db_path = "./airflow/data/tipsterdeals.duckdb"
tables = check_table_name(db_path)
print("Tables in the database:", tables)

Tables in the database: []
